# Importation des librairies

In [2]:
# Import des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import ttest_ind, chi2_contingency, f_oneway, skew, kurtosis

# Configuration des visualisations
sns.set_style("whitegrid")
sns.set_palette('pastel')

type_colors = {
    'Normal': '#A8A878',    # Beige
    'Fire': '#F08030',      # Orange-red
    'Water': '#6890F0',     # Blue
    'Electric': '#F8D030',  # Yellow
    'Grass': '#78C850',     # Green
    'Ice': '#98D8D8',       # Light blue
    'Fighting': '#C03028',  # Dark red
    'Poison': '#A040A0',    # Purple
    'Ground': '#E0C068',    # Brown
    'Flying': '#A890F0',    # Light purple
    'Psychic': '#F85888',   # Pink
    'Bug': '#A8B820',       # Light green
    'Rock': '#B8A038',      # Dark brown
    'Ghost': '#705898',     # Purple-grey
    'Dragon': '#7038F8',    # Dark blue-purple
    'Dark': '#705848',      # Dark grey-brown
    'Steel': '#B8B8D0',     # Light grey
    'Fairy': '#EE99AC'      # Light pink
}

# 🧼 Chargement et aperçu du jeu de données

In [14]:
df = pd.read_csv("data/MP-24-25_clean.csv")
df.shape
df.head()

,pokedex_number,name,status,species,height,weight,abilities_number,ability_1,ability_2,ability_hidden,...,against_dark,against_steel,against_fairy,generation,type_number,type_1,type_2,egg_type_number,egg_type_1,egg_type_2
0,1,Bulbasaur,Normal,Seed Pokémon,0.71,6.9,2,Overgrow,NaN,Chlorophyll,...,1.0,1.0,0.5,1,2,Grass,Poison,2,Grass,Monster
1,2,Ivysaur,Normal,Seed Pokémon,0.99,13.0,2,Overgrow,NaN,Chlorophyll,...,1.0,1.0,0.5,1,2,Grass,Poison,2,Grass,Monster
2,3,Venusaur,Normal,Seed Pokémon,2.01,100.0,2,Overgrow,NaN,Chlorophyll,...,1.0,1.0,0.5,1,2,Grass,Poison,2,Grass,Monster
3,3,Mega Venusaur,Normal,Seed Pokémon,2.39,155.5,1,Thick Fat,NaN,NaN,...,1.0,1.0,0.5,1,2,Grass,Poison,2,Grass,Monster
4,4,Charmander,Normal,Lizard Pokémon,0.61,8.5,2,Blaze,NaN,Solar Power,...,1.0,0.5,0.5,1,1,Fire,NaN,2,Dragon,Monster


# 🧽 Nettoyage et préparation
  - Suppression des colonnes inutiles (Unnamed, german_name, japanese_name, etc.)

  - Conversion des tailles et poids

  - Extraction des types

  - Création d’une colonne generation

> 🔎 Ces étapes ont déjà été réalisées dans le projet 4EDAP. Nous utiliserons la version df_clean.

# 📊 1. Analyse exploratoire ciblée
  - Analyse des distributions de status et hp

  - Corrélations avec les autres variables

  - Visualisations : boxplots, histogrammes, heatmap, etc.